In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time
from time import time as timer

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

/home/cfchen/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    'gpu': 0.85
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.04858541488647461s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

## With Video Resizing

In [4]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('temp.avi',fourcc, 30.0, (960, 540))     # resize image half

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

start = timer()

count = 0
while(True):
    ret, frame = capture.read() 

    if ret == True:
        
        count += 1
        
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
                    
        results = tfnet.return_predict(frame)
        
        print('Frame Number {}: Found {} boxes for {}'.format(count - 1, len(results), 'img'))
        #print(results)
        
        for result in results:
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 2)
            frame = cv2.rectangle(frame, tl, br, (255, 255, 0), 7)
            frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 
                                0.65, (0, 255, 255), 2)
            
            Label_Confidence = '{} {:.2f}'.format(result['label'], result['confidence'])
            TLXY = '{} {}'.format(result['topleft']['x'], result['topleft']['y'])
            BRXY = '{} {}'.format(result['bottomright']['x'] , result['bottomright']['y'])
            
            print("{}  {}  {} {}".format(count - 1 , Label_Confidence, TLXY, BRXY))
        
        out.write(frame)
        cv2.imshow('frame', frame)

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

Video Dim: 1920.0  X  1080.0
Frame Number 0: Found 8 boxes for img
0  car 0.64  530 50 576 81
0  truck 0.43  445 64 516 129
0  car 0.24  684 101 760 150
0  car 0.42  690 133 747 182
0  car 0.45  58 186 158 253
0  car 0.49  702 176 774 255
0  truck 0.40  238 70 334 122
0  truck 0.25  150 100 231 158
Frame Number 1: Found 10 boxes for img
1  car 0.21  564 7 590 22
1  car 0.20  513 29 544 45
1  car 0.50  531 51 575 81
1  car 0.51  690 133 746 183
1  car 0.46  57 186 158 254
1  car 0.53  701 177 775 254
1  truck 0.50  442 65 515 132
1  truck 0.48  239 68 333 122
1  truck 0.27  147 100 233 159
1  traffic light 0.21  532 40 585 84
Frame Number 2: Found 10 boxes for img
2  car 0.21  511 31 543 47
2  car 0.61  529 53 576 82
2  car 0.24  684 101 758 152
2  car 0.51  690 132 747 185
2  car 0.49  701 176 775 253
2  car 0.50  66 198 163 266
2  truck 0.47  238 70 331 122
2  truck 0.59  440 67 513 135
2  truck 0.29  146 101 233 162
2  traffic light 0.20  251 140 266 163
Frame Number 3: Found 9 boxes

Frame Number 34: Found 9 boxes for img
34  car 0.28  519 26 550 42
34  car 0.29  478 72 541 134
34  car 0.49  689 132 748 183
34  car 0.53  55 184 160 253
34  car 0.45  699 180 775 252
34  car 0.25  0 216 57 320
34  bus 0.44  329 113 445 219
34  truck 0.37  240 66 331 122
34  truck 0.38  330 116 448 230
Frame Number 35: Found 9 boxes for img
35  car 0.29  519 27 551 42
35  car 0.45  476 72 541 135
35  car 0.50  689 131 748 183
35  car 0.53  56 184 160 253
35  car 0.46  699 179 774 252
35  car 0.26  0 215 57 320
35  bus 0.21  322 110 444 224
35  truck 0.39  239 66 331 121
35  truck 0.61  321 119 440 230
Frame Number 36: Found 7 boxes for img
36  car 0.24  520 26 551 42
36  car 0.56  473 73 544 138
36  car 0.55  688 131 749 183
36  car 0.54  54 181 161 256
36  car 0.39  700 180 774 252
36  truck 0.37  238 64 331 122
36  truck 0.64  318 121 437 232
Frame Number 37: Found 7 boxes for img
37  car 0.25  519 26 551 43
37  car 0.53  471 72 544 140
37  car 0.54  688 131 748 183
37  car 0.54  54

Frame Number 68: Found 8 boxes for img
68  car 0.48  689 133 750 185
68  car 0.64  49 182 162 256
68  car 0.38  702 180 776 253
68  car 0.31  0 213 59 321
68  truck 0.48  383 126 495 247
68  truck 0.25  61 241 266 473
68  bus 0.34  64 266 265 486
68  truck 0.26  243 67 332 122
Frame Number 69: Found 8 boxes for img
69  car 0.48  689 133 750 184
69  car 0.63  50 183 161 255
69  car 0.29  364 133 495 270
69  car 0.38  702 180 776 253
69  car 0.30  0 212 59 322
69  bus 0.39  58 266 259 491
69  truck 0.26  243 67 331 122
69  truck 0.49  375 132 486 257
Frame Number 70: Found 9 boxes for img
70  car 0.21  520 27 550 42
70  car 0.49  689 133 750 185
70  car 0.63  51 184 160 254
70  car 0.29  361 138 489 269
70  car 0.38  702 180 777 253
70  car 0.26  0 213 59 322
70  bus 0.66  43 264 247 499
70  truck 0.31  243 67 332 122
70  truck 0.45  370 133 482 260
Frame Number 71: Found 9 boxes for img
71  car 0.49  690 133 749 184
71  car 0.62  51 184 160 254
71  car 0.31  359 140 486 272
71  car 0.38

Frame Number 104: Found 6 boxes for img
104  person 0.31  532 229 603 429
104  car 0.55  690 132 749 183
104  car 0.53  56 183 160 255
104  car 0.51  703 182 779 249
104  car 0.25  0 211 57 324
104  truck 0.35  243 66 336 122
Frame Number 105: Found 6 boxes for img
105  person 0.58  556 216 586 279
105  car 0.55  690 132 749 183
105  car 0.53  56 184 160 255
105  car 0.51  704 183 779 248
105  car 0.25  0 212 57 324
105  truck 0.37  244 66 336 122
Frame Number 106: Found 6 boxes for img
106  person 0.48  557 210 588 270
106  car 0.53  690 132 749 183
106  car 0.53  56 184 160 254
106  car 0.51  705 183 778 248
106  car 0.26  0 212 57 323
106  truck 0.37  243 66 336 122
Frame Number 107: Found 8 boxes for img
107  person 0.52  562 198 591 240
107  person 0.44  547 202 592 336
107  car 0.55  690 132 749 183
107  car 0.53  55 184 160 254
107  car 0.53  704 183 778 249
107  car 0.26  0 213 57 323
107  truck 0.37  243 66 336 122
107  cell phone 0.22  28 332 328 535
Frame Number 108: Found 9

Frame Number 134: Found 7 boxes for img
134  traffic light 0.33  592 87 613 128
134  car 0.20  519 29 544 42
134  car 0.52  688 131 750 181
134  car 0.57  53 184 161 255
134  car 0.52  703 181 779 252
134  car 0.26  0 215 56 321
134  truck 0.33  242 65 335 122
Frame Number 135: Found 7 boxes for img
135  person 0.32  592 87 613 125
135  car 0.23  518 29 545 43
135  car 0.52  688 132 751 181
135  car 0.57  54 184 161 255
135  car 0.52  702 181 779 252
135  car 0.25  0 215 56 321
135  truck 0.34  242 65 335 122
Frame Number 136: Found 7 boxes for img
136  traffic light 0.26  593 85 613 124
136  car 0.22  518 28 545 42
136  car 0.51  688 132 751 181
136  car 0.58  53 183 161 255
136  car 0.51  702 181 779 252
136  car 0.27  0 214 56 322
136  truck 0.35  243 65 335 121
Frame Number 137: Found 7 boxes for img
137  person 0.29  594 84 613 123
137  car 0.22  518 28 546 43
137  car 0.53  688 132 751 182
137  car 0.58  53 183 161 255
137  car 0.51  702 180 779 252
137  car 0.26  0 214 57 322
13

Frame Number 176: Found 5 boxes for img
176  car 0.20  520 29 545 44
176  car 0.50  689 133 749 182
176  car 0.57  54 183 161 256
176  car 0.43  702 181 779 252
176  truck 0.33  238 65 333 122
Frame Number 177: Found 5 boxes for img
177  car 0.21  520 29 546 43
177  car 0.49  689 133 749 182
177  car 0.56  54 183 161 256
177  car 0.42  701 180 779 253
177  truck 0.34  238 64 334 122
Frame Number 178: Found 4 boxes for img
178  car 0.49  689 133 749 183
178  car 0.57  54 183 161 256
178  car 0.42  701 180 779 252
178  truck 0.33  237 65 334 122
Frame Number 179: Found 4 boxes for img
179  car 0.50  689 133 749 183
179  car 0.56  55 183 161 255
179  car 0.43  702 180 779 253
179  truck 0.34  238 65 334 122
Frame Number 180: Found 4 boxes for img
180  car 0.52  689 133 750 182
180  car 0.58  53 182 161 256
180  car 0.47  702 182 779 251
180  truck 0.32  241 66 333 122
Frame Number 181: Found 4 boxes for img
181  car 0.53  689 133 750 182
181  car 0.58  54 182 161 256
181  car 0.47  702 18

Frame Number 224: Found 4 boxes for img
224  car 0.47  691 133 750 183
224  car 0.55  53 184 162 255
224  car 0.41  702 180 778 253
224  truck 0.35  243 65 333 122
Frame Number 225: Found 4 boxes for img
225  car 0.47  691 132 750 183
225  car 0.53  53 183 162 255
225  car 0.41  702 180 778 252
225  truck 0.38  244 65 333 121
Frame Number 226: Found 4 boxes for img
226  car 0.48  691 132 750 183
226  car 0.55  53 184 162 255
226  car 0.41  702 180 778 252
226  truck 0.33  244 65 333 121
Frame Number 227: Found 4 boxes for img
227  car 0.46  691 132 750 183
227  car 0.55  53 183 161 255
227  car 0.42  701 181 778 252
227  truck 0.35  244 64 334 121
Frame Number 228: Found 5 boxes for img
228  car 0.21  520 28 549 43
228  car 0.44  690 131 751 183
228  car 0.59  53 184 161 257
228  car 0.43  701 180 777 251
228  truck 0.33  243 65 337 123
Frame Number 229: Found 6 boxes for img
229  car 0.20  520 28 548 43
229  car 0.20  683 101 759 155
229  car 0.44  690 131 751 183
229  car 0.60  52 18